# Image Watermarking: Examples

This notebook shows how we can use Omniseal Bench Python APIs to watermark image with different models, and to apply attacks then detect the watermarks and print out different metrics

## Example 1: Quick start

In the following example, we load a custom local dataset, then use public trustmark model to watermark each image. Next, we run the attacks and detect if watermarks surive them. Finally, we report the quality metrics and detection scores.

In [1]:
# Some Jupyter kernels check TORCH_DISTRIBUTED_DEBUG and throw an error if not found
import os
os.environ["TORCH_DISTRIBUTED_DEBUG"] = "OFF"

In [2]:
import sys
sys.executable

'/private/home/tuantran/.conda/envs/omnisealbench/bin/python'

In [3]:
import shutil
from pathlib import Path

# This notebook runs inside the root directory of omnisealbench. Adjust this when you run in a different location
example_image_dir = "examples/img"

def build_test_image_dataset():
    # Make up a fake local dataset from test image
    example_image = "examples/img/img_1.png"

    tmp_path = Path("tmp/test_images")
    tmp_path.mkdir(exist_ok=True, parents=True)

    shutil.copy(example_image, tmp_path / "img_1_1.png")
    shutil.copy(example_image, tmp_path / "img_1_2.png")
    shutil.copy(example_image, tmp_path / "img_1_3.png")
    shutil.copy(example_image, tmp_path / "img_1_4.png")
    shutil.copy(example_image, tmp_path / "other_random_image_1.png")
    shutil.copy(example_image, tmp_path / "other_random_image_2.png")
    shutil.copy(example_image, tmp_path / "other_random_image_3.png")
    shutil.copy(example_image, tmp_path / "other_random_image_4.png")
    
build_test_image_dataset()

In [ ]:
from omnisealbench import AttackConfig, task, get_model

attacks = [
    AttackConfig(name="jpeg", data_type="image", args={"quality_factor": 50}),
    AttackConfig(name="gaussian_blur", data_type="image", args={"kernel_size": 13}),
]

e2e = task(
    "default",
    modality="image",
    
    # Dataset options
    dataset_dir="tmp/test_images",
    
    # data loading options
    batch_size=2,
    num_workers=2,
    
    # watermark options:
    # message_size (Optional): THis is only specified if the model does not have the attribute 'nbits'
    how_to_generate_message="per_dataset",  # Options: 'per_dataset', 'per_batch'
    
    # detection options
    detection_bits=16,  # how many leading bits are reserved to calculate the detection accuracy etc.
    attacks=attacks,

    seed=42,

    # Put `metrics="all"` for computing all metrics ("ssim", "lpips", "psnr")
    metrics=["ssim", "lpips"],

)

model = get_model("trustmark", device="cuda")

avg_results, raw_results = e2e(model)

omnisealbench.models.trustmark.build_model has the following dependencies: ['trustmark']
Initializing TrustMark watermarking with ECC using [cuda]


/private/home/tuantran/.conda/envs/omnisealbench/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running ImageWatermarkAttacksAndDetection with attack: no-attack


/private/home/tuantran/.conda/envs/omnisealbench/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/private/home/tuantran/.conda/envs/omnisealbench/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Running ImageWatermarkAttacksAndDetection with attack: jpeg__quality_factor_50


/private/home/tuantran/.conda/envs/omnisealbench/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/private/home/tuantran/.conda/envs/omnisealbench/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Running ImageWatermarkAttacksAndDetection with attack: gaussian_blur__kernel_size_13


/private/home/tuantran/.conda/envs/omnisealbench/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/private/home/tuantran/.conda/envs/omnisealbench/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


### Explanation:

In the above example, we load a local image directory, then create a task object `e2e` and model object `model`, follow by loading the model and running `__call__()` function of `e2e` to perform the end-to-end evaluation. The result is a tuple of 2 items:

1) Average metrics: For each metric, we compute the average score over the evaluation data, and return:
    - `avg`: mean value
    - `count`: Number of items
    - `square`: Mean of the squares of the values
    - `avg_ci_fn`: A function applied to the bounds of the confidence interval

2) Raw scores: For each attack, we have a dictionary of a metric scores, while each score, we have the full list of `n` individual scores.

In [7]:
avg_results

{'watermark_det_score': AverageMetric(avg=1.0, count=24, square=1.0, avg_ci_fn=None),
 'watermark_det': AverageMetric(avg=1.0, count=24, square=1.0, avg_ci_fn=None),
 'fake_det_score': AverageMetric(avg=0.0, count=24, square=0.0, avg_ci_fn=None),
 'fake_det': AverageMetric(avg=0.0, count=24, square=0.0, avg_ci_fn=None),
 'bit_acc': AverageMetric(avg=1.0, count=24, square=1.0, avg_ci_fn=None),
 'word_acc': AverageMetric(avg=1.0, count=24, square=1.0, avg_ci_fn=None),
 'p_value': AverageMetric(avg=9.094947017729282e-13, count=24, square=8.271806125530277e-25, avg_ci_fn=None),
 'capacity': AverageMetric(avg=40.0, count=24, square=1600.0, avg_ci_fn=None),
 'log10_p_value': AverageMetric(avg=-12.041199826559248, count=24, square=144.99049326313047, avg_ci_fn=None),
 'ssim': AverageMetric(avg=0.995694120724996, count=24, square=0.9914150635401408, avg_ci_fn=None),
 'lpips': AverageMetric(avg=0.026802892486254375, count=24, square=0.0008639693652569535, avg_ci_fn=None),
 'decoder_time': Avera

Above we have 8 data items, 3 attacks (the default "no-attack" is added internally and will always be run), so 24 items in total. `avg_results` has 15 metrics, falling in:

- Quality metrics: Defined by `metrics` ("ssim", "lpips")

- Profiling time: "qual_ime" (average time to run quality metrics), "det_timne" (average time to run detection), "attack_time" (average time to run attacks), "decoder_time" (average time to decode messages)

- Detection scores: (All remaining scores)

In [8]:
len(raw_results)

3

In [9]:
raw_results[0]

defaultdict(list,
            {'watermark_det_score': [1, 1, 1, 1, 1, 1, 1, 1],
             'watermark_det': [True, True, True, True, True, True, True, True],
             'fake_det_score': [0, 0, 0, 0, 0, 0, 0, 0],
             'fake_det': [False,
              False,
              False,
              False,
              False,
              False,
              False,
              False],
             'bit_acc': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
             'word_acc': [True, True, True, True, True, True, True, True],
             'p_value': [9.094947017729282e-13,
              9.094947017729282e-13,
              9.094947017729282e-13,
              9.094947017729282e-13,
              9.094947017729282e-13,
              9.094947017729282e-13,
              9.094947017729282e-13,
              9.094947017729282e-13],
             'capacity': [40.0, 40.0, 40.0, 40.0, 40.0, 40.0, 40.0, 40.0],
             'log10_p_value': [-12.041199826559248,
              -12.04119982

Here `raw_results` has 3 dictionary for each attack ("no-attack" and 2 defined attacks). For each dictionary, we have 15 keys corresponding to 15 metrics, and each value is a list of 8 scores for 8 items

<hr/>

### Concepts:
#### Task:

Omniseal Bench Python API consists of 2 main building blocks: **Task** and **Model**. The Task defines the dataset to be used for the evaluation, how to load data (batch size, padding, etc.) and metrics to run. 

To construct the task, we use `omnisealbench.task()`. To construct the task, you must specify at least 2 arguments: Task type and modality. The modality can be "audio", "image", "video". The task type can be one of the three:

- **generation**: Run a generator over specific dataset to watermark each of its items, and return the quality metrics of the watermarked audios.

- **detection**: Apply a specific set of attacks over a watermarked dataset, then run detector and report the detection scores (robustness) and as well as quality metrics for each attack.

- **default**: Run end-to-end genration and detection.

You can also look all available tasks with `omnisealbench.list_tasks` API:

In [6]:
from omnisealbench import list_tasks

list_tasks()

- "('generation', 'audio')": Running audio watermarking on a HuggingFace dataset and evaluating the quality of the generated watermarks. (class/function: omnisealbench.tasks.audio.evaluate_audio_watermark_generation)
- "('detection', 'audio')": Running audio watermarking on a custom dataset and evaluating the quality and robustness of the generated watermarks. (class/function: omnisealbench.tasks.audio.evaluate_audio_watermark_attacks_and_detection)
- "('default', 'audio')": Running audio watermarking end-to-end (class/function: omnisealbench.tasks.audio.evaluate_audio_watermark_end2end)
- "('default', 'image')": Running image watermarking end-to-end (class/function: omnisealbench.tasks.image.evaluate_image_watermark_end2end)
- "('generation', 'image')": Running image watermarking on a local dataset and evaluating the quality of the generated watermarks. (class/function: omnisealbench.tasks.image.evaluate_image_watermark_generation)
- "('detection', 'image')": Running image watermarkin

Each task expects a specific list of arguments, falling into three categories:
- _input data_: Specify how dataset is loaded into the task. Note that the dataset is only defined at the task definition task, the actual data loading happens when we execute the task.

- _execution_: Define the behaviour of the execution such as metrics, attacks, device, seed for reproducibility, 

- _output handling_: Define how the results are handled. These includes result directory and filename, whether to keep intermediate data in cache, whether we should log details some small subset of data for inspection, etc.


To get list of arguments for a task, call "explain()":

In [10]:
from omnisealbench import explain

print(explain("default", modality="image"))

#### Model

A model is a wrapper over the watermarking model, that converts the watermarking output to the format that the task understands. Please refer to the notebook [Model.ipynb](Model.ipynb) for more detailed instructions.

Omniseal Bench provides some ready-to-use wrapper for the popular Audio watermarking models: 
- [TrustMark](https://github.com/adobe/trustmark)
- [CIN](https://github.com/rmpku/CIN)
- [MBRS](https://github.com/jzyustc/MBRS)
- [DCT-DWT](https://github.com/ShieldMnt/invisible-watermark)
- [FNNS](https://github.com/varshakishore/FNNS)
- [InvisMark](https://github.com/ShieldMnt/invisible-watermark)
- [SSL](https://github.com/ShieldMnt/invisible-watermark)
- [WAM]https://github.com/facebookresearch/watermark-anything

The models are registered via model cards in "src/omnisealbench/cards". To get the model, we call `get_model()` and pass the name of the model card (e.g. "wavmark_fast"), plus any free argument to override the attributes defined in the card.


#### Attacks:

For each modality, we have implemented different attacks in `omnisealbench.attacks`. The attack can be further parameterized to different _attack variant_, with concrete function parameter values. 

For convenience, we provide some fixed set of attacks and attack variants in the _attack registry_, which is a YAML file defining how to create the attack and attack variants. These registries can be found in "src/omnisealbench/attacks" directory (For example, "src/omnisealbench/attacks/image_attacks.yaml").


For more details on attacks, see the notebook [Attacks](Attacks.ipynb).

## Example 2: Caching and saving data

By default the API will not save the final result and data. To save the data to external files for further inspection, we pass the "cache_dir" and "result_dir":

In [9]:
# We run the notebook within the root directory, adjust the directory path
from omnisealbench import task, get_model
import warnings

# Suppress all warnings in this cell
warnings.filterwarnings("ignore")

e2e = task(
    "default",
    modality="image",
    
    # Dataset options
    dataset_dir="../examples/img",
    
    # data loading options
    batch_size=2,
    num_workers=2,
    
    # watermark options:
    # message_size (Optional): THis is only specified if the model does not have the attribute 'nbits'
    how_to_generate_message="per_dataset",  # Options: 'per_dataset', 'per_batch'
    
    # detection options
    detection_bits=16,  # how many leading bits are reserved to calculate the detection accuracy etc.
    attacks="../tests/image/image_attacks_mini.yaml",

    cache_dir="watermark_img_results",
    result_dir="final_img_results",
    overwrite=True,
    ids_to_save="all",  # Save all audios

    seed=42,
    metrics=["ssim", "lpips"],
)

model = get_model("trustmark", device="cuda")

avg_results, raw_results = e2e(model)


omnisealbench.models.trustmark.build_model has the following dependencies: ['trustmark']
Initializing TrustMark watermarking with ECC using [cuda]


Result watermark_img_results exists and will be overriden as --overwrite is set
Result final_img_results exists and will be overriden as --overwrite is set
Running ImageWatermarkAttacksAndDetection with attack: no-attack
Running ImageWatermarkAttacksAndDetection with attack: proportion__prop_0.5
Running ImageWatermarkAttacksAndDetection with attack: collage__scale_1.0
Running ImageWatermarkAttacksAndDetection with attack: center_crop__scale_0.05
Running ImageWatermarkAttacksAndDetection with attack: jpeg__quality_factor_90
Running ImageWatermarkAttacksAndDetection with attack: sharpness__sharpness_factor_0.25
Running ImageWatermarkAttacksAndDetection with attack: resize__scale_0.89
Running ImageWatermarkAttacksAndDetection with attack: overlay_text
Running ImageWatermarkAttacksAndDetection with attack: hflip
Running ImageWatermarkAttacksAndDetection with attack: perspective__distortion_scale_0.4
Running ImageWatermarkAttacksAndDetection with attack: median_filter__kernel_size_3
Running

In [10]:
import os
def print_directory_tree(start_path, indent=""):
    for item in os.listdir(start_path):
        path = os.path.join(start_path, item)
        if os.path.isdir(path):
            print(f"{indent}📁 {item}/")
            print_directory_tree(path, indent + "    ")
        else:
            print(f"{indent}📄 {item}")

            
print_directory_tree("final_img_results")

📁 no-attack/
    📄 attack_results.jsonl
    📄 metrics.json
📁 proportion__prop_0.5/
    📄 attack_results.jsonl
    📄 metrics.json
📁 collage__scale_1.0/
    📄 attack_results.jsonl
    📄 metrics.json
📁 center_crop__scale_0.05/
    📄 attack_results.jsonl
    📄 metrics.json
📁 jpeg__quality_factor_90/
    📄 attack_results.jsonl
    📄 metrics.json
📁 sharpness__sharpness_factor_0.25/
    📄 attack_results.jsonl
    📄 metrics.json
📁 resize__scale_0.89/
    📄 attack_results.jsonl
    📄 metrics.json
📁 overlay_text/
    📄 attack_results.jsonl
    📄 metrics.json
📁 hflip/
    📄 attack_results.jsonl
    📄 metrics.json
📁 perspective__distortion_scale_0.4/
    📄 attack_results.jsonl
    📄 metrics.json
📁 median_filter__kernel_size_3/
    📄 attack_results.jsonl
    📄 metrics.json
📁 median_filter__kernel_size_5/
    📄 attack_results.jsonl
    📄 metrics.json
📁 hue__hue_factor_0.2/
    📄 attack_results.jsonl
    📄 metrics.json
📁 gaussian_blur__kernel_size_17/
    📄 attack_results.jsonl
    📄 metrics.json
📁 c

### Explanation: 

- Instead of passing attacks name, we can pass a custom attack registry file to the `attacks` argument.

- The final results are stored in the directory specified by `result_dir`. This consists of several sub-directories, each correspond to a task. In the above example, the YAML file "audio_attacks_mini.yaml" consists of 6 attacks, so we have 7 sub-directories (including the default "no-attack") under the result directory.

- For each task, we also have average metrics and raw results, similar to Example 1:
   - The average metrics are saved to the file "metrics.json". For each metric, the value is saved as a list of three values "avg", "count", "square", corresponding to the object `AverageMetric`. 
   - The raw results are saved to the file "attacks_results.jsonl". Since this is saved per attack, we have only one JSON line for each sub-directory.

- `cache_dir`: The watermark is only generated one time, and the output will be saved to `cache_dir` and are used by detection tasks of all attacks.

- `overwrite`: Whether we should overwrite all existing files and results in the `result_dir` and `cache_dir`. By default, if the task finds these existing results, it will just load them and skip the execution.

- `ids_to_save`: Choose which items to save. We can define to save all data to hard disk ("all"), or a list of specific ids ("0,1,2"), or a range of ids ("0-3") or mixed of them ("0,2,3,4-8").

### Cache:

> ⚠️ **Warning:** Right now the watermark generation WILL ALWAYS be cached before running the detection. If `cache_dir` is not specified, Omniseal Bench will use the default cache location in "~/.cache/omniseal/tmp" instead. The consequence is the directory size might grow significantly over time. 
We are implementing the "streaming" fashion ("keep_data_in_memory") to support quick analysis scenario on small dataset, where no data is cached. THis feature will be added soon, but until then, consider cleaning the cache after a while to save storage

### Final report:

The final report is saved to "report.csv" which consists of average metrics over all attack variants. This can also be obtained programmatically with "print_scores()" API:

In [12]:
report = e2e.print_scores(raw_results)
report

,watermark_det_score,watermark_det,fake_det_score,fake_det,bit_acc,word_acc,p_value,capacity,log10_p_value,ssim,lpips,decoder_time,qual_time,det_time,attack_time,idx,attack,attack_variant,cat
0,1,True,0,False,1.000,True,9.094947e-13,40.000000,-12.041200,0.997072,0.019420,0.0229,2.0030,0.0024,0.0000,0,no-attack,default,none
1,1,True,0,False,1.000,True,9.094947e-13,40.000000,-12.041200,NaN,NaN,0.0244,0.0000,0.0017,0.0006,0,proportion,prop_0.5,Geometric
2,1,True,0,False,1.000,True,9.094947e-13,40.000000,-12.041200,0.930317,0.116886,0.0215,1.9915,0.0016,0.0015,0,collage,scale_1.0,Inpainting
3,0,False,0,False,0.425,False,8.659064e-01,0.651670,-0.062529,NaN,NaN,0.0173,0.0000,0.0018,0.0004,0,center_crop,scale_0.05,Geometric
4,1,True,0,False,1.000,True,9.094947e-13,40.000000,-12.041200,0.993528,0.030834,0.0221,1.9929,0.0015,0.0125,0,jpeg,quality_factor_90,Compression
5,1,True,0,False,1.000,True,9.094947e-13,40.000000,-12.041200,0.997271,0.019720,0.0227,1.9766,0.0017,0.0011,0,sharpness,sharpness_factor_0.25,Visual
6,1,True,0,False,1.000,True,9.094947e-13,40.000000,-12.041200,NaN,NaN,0.0225,0.0000,0.0016,0.0005,0,resize,scale_0.89,Geometric
7,1,True,0,False,1.000,True,9.094947e-13,40.000000,-12.041200,0.997150,0.017451,0.0206,1.9870,0.0014,0.0125,0,overlay_text,default,Inpainting
8,1,True,0,False,1.000,True,9.094947e-13,40.000000,-12.041200,NaN,NaN,0.0236,0.0000,0.0016,0.0003,0,hflip,default,Geometric
9,0,False,0,False,0.400,False,9.230700e-01,1.161976,-0.034765,NaN,NaN,0.0222,0.0000,0.0014,0.0019,0,perspective,distortion_scale_0.4,Geometric


## Example 3: Generating watermarks only

If you want to generate watermarks only (and optionally look at the quality metrics) without running any attacks, use the "generation" task:

In [14]:
# In some CUDAs in FAIR cluster, calling AudioSeal compile() results in torch dynamo error, so we disable compile() with NO_TORCH_COMPILE
os.environ["NO_TORCH_COMPILE"] = "1"

generation_task = task(
    "generation",
    modality="image",
    # Dataset options
    dataset_dir="../examples/img",
    
    # data loading options
    batch_size=2,
    num_workers=2,
    
    # watermark options:
    # message_size (Optional): THis is only specified if the model does not have the attribute 'nbits'
    how_to_generate_message="per_dataset",  # Options: 'per_dataset', 'per_batch'
    
    result_dir="watermark_img_results",
    overwrite=True,
    ids_to_save="all",  # Save all audios

    seed=42,
    metrics=["ssim", "lpips"],
)

generator = get_model("dct_dwt", as_type="generator", device="cuda")

avg_metrics, raw_results = generation_task(generator)

omnisealbench.models.dctdwt.build_generator has the following dependencies: ['invisible-watermark']


Result watermark_img_results exists and will be overriden as --overwrite is set



NOTE: We pass `as_type` argument to the `get_model()` call to indicate that we only load the generator checkpoint from the model. See [Model](Model.ipynb) for more details.

In [15]:
print_directory_tree("watermark_img_results")

📁 watermarks/
    📄 watermarked_00000.png
    📄 message_0.txt
📄 watermark_results.jsonl
📄 metrics.json


The content of `avg_metrics` and `raw_results` are saved to two files `metrics.json` and `watermark_results.jsonl` accordingly.

For each of the saved items, we save the watermarked audio and secret message separately. Because we specify `how_to_generate_message="per_dataset"`, all "message_*.txt" should contain the same secret binary string.

If we want to save the original data too, specify the path in `data_subdir`:

In [16]:
generation_task_2 = task(
    "generation",
    modality="image",
    # Dataset options
    dataset_dir="../examples/img",
    
    # data loading options
    batch_size=2,
    num_workers=2,
    
    # watermark options:
    # message_size (Optional): THis is only specified if the model does not have the attribute 'nbits'
    how_to_generate_message="per_dataset",  # Options: 'per_dataset', 'per_batch'
    
    result_dir="watermark_img_results",
    data_subdir="data",
    overwrite=True,
    ids_to_save="all",  # Save all audios

    seed=42,
    metrics=["ssim", "lpips"],
)

generation_task_2(generator)

Result watermark_img_results exists and will be overriden as --overwrite is set


({'ssim': AverageMetric(avg=0.9799583554267883, count=1, square=0.960318386554718, avg_ci_fn=None),
  'lpips': AverageMetric(avg=0.024358447641134262, count=1, square=0.0005933339714858793, avg_ci_fn=None),
  'qual_time': AverageMetric(avg=2.069, count=1, square=4.280761, avg_ci_fn=None)},
 defaultdict(list,
             {'ssim': [0.97995836],
              'lpips': [0.024358447641134262],
              'qual_time': [2.069],
              'idx': [0]}))

In [17]:
print_directory_tree("watermark_img_results")

📁 watermarks/
    📄 watermarked_00000.png
    📄 message_0.txt
📁 data/
    📄 original_00000.png
📄 watermark_results.jsonl
📄 metrics.json


Note the new sub-directory "data" with original image

## Example 4: Detection only
This scenario is useful when we just want to evaluate the model that generate the watermarks externally, for example the latent watermarking model.


In [22]:
detection_task = task(
    "detection",
    modality="image",
    seed=42,
    dataset_dir="watermark_img_results",
    original_image_pattern="data/*.png",
    watermarked_image_pattern="watermarks/*.png",  # We just fake the watermarked images
    message_pattern="watermarks/message_*.txt",
    metrics=["psnr"],  # We can add one more quality metric that was not previously computed
    result_dir="detection_image",
    overwrite=True,
    attacks=["comb", "gaussian_blur"],
    batch_size=2,
)

detector = get_model("dct_dwt", as_type="detector", device="cuda")

avg_metrics, raw_results = detection_task(detector)

omnisealbench.models.dctdwt.build_detector has the following dependencies: ['invisible-watermark']


Running ImageWatermarkAttacksAndDetection with attack: no-attack
Running ImageWatermarkAttacksAndDetection with attack: gaussian_blur__kernel_size_3
Running ImageWatermarkAttacksAndDetection with attack: gaussian_blur__kernel_size_5
Running ImageWatermarkAttacksAndDetection with attack: gaussian_blur__kernel_size_9
Running ImageWatermarkAttacksAndDetection with attack: gaussian_blur__kernel_size_13
Running ImageWatermarkAttacksAndDetection with attack: gaussian_blur__kernel_size_17
Skipping attack omnisealbench.attacks.image_effects.comb for ImageWatermarkAttacksAndDetection on DCTDWTDetector


In [23]:
detection_task.print_scores(raw_results)

,watermark_det_score,watermark_det,fake_det_score,fake_det,bit_acc,word_acc,p_value,capacity,log10_p_value,psnr,decoder_time,qual_time,det_time,attack_time,idx,attack,attack_variant,cat
0,0.75,True,0.75,True,0.34375,False,0.974949,2.292416,-0.011018,38.669281,0.0304,0.0008,0.0022,0.0000,0.0,no-attack,default,none
1,0.75,True,0.75,True,0.34375,False,0.974949,2.292416,-0.011018,44.933609,0.0296,0.0005,0.0015,0.0012,0.0,gaussian_blur,kernel_size_3,Visual
2,0.75,True,0.75,True,0.34375,False,0.974949,2.292416,-0.011018,47.305355,0.0318,0.0005,0.0014,0.0009,0.0,gaussian_blur,kernel_size_5,Visual
3,0.75,True,0.75,True,0.34375,False,0.974949,2.292416,-0.011018,50.207535,0.0297,0.0005,0.0015,0.0009,0.0,gaussian_blur,kernel_size_9,Visual
4,0.75,True,0.75,True,0.34375,False,0.974949,2.292416,-0.011018,51.915569,0.0297,0.0005,0.0014,0.0009,0.0,gaussian_blur,kernel_size_13,Visual
5,0.75,True,0.75,True,0.34375,False,0.974949,2.292416,-0.011018,52.994637,0.0297,0.0004,0.0015,0.0008,0.0,gaussian_blur,kernel_size_17,Visual
